##Exercise

The goal of this exercise is to combine the scoring and assists statistics for every player in the NBA in 2014-2015.  The end result will have them in a pandas dataframe with the fields from both pages for every player.

In [1]:
from bs4 import BeautifulSoup
from urllib2 import urlopen
from collections import OrderedDict
import pandas as adorable

URL_FORMAT_STRING = 'http://espn.go.com/nba/statistics/player/_/stat/%s/qualified/false'
SCORE_URL = URL_FORMAT_STRING % 'scoring-per-game'
ASSIST_URL = URL_FORMAT_STRING % 'assists'

The general steps should be as follows:

- Create a function get_cols that retrieves the names of the header columns given a table element (skip the ranks, split the positions)

In [2]:
# parse the column names into a list.
def parse_column_headers (table):
    header_cells = table.find(attrs={'class':'colhead'}).findAll('td')
    column_names = []
    for cell in header_cells:
        # skip rank
        if (cell.string == 'RK'):
            continue
        column_names.append(cell.string)
        # The column called PLAYER really has 2 fields of information,
        # the player and his position. When we encounter PLAYER,
        # push an additional column called POSITION on there.
        if (cell.string == 'PLAYER'):
            column_names.append(u'POSITION')
    return column_names

- Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  You can use either the dict approach or the numpy array approach.

In [3]:
# parse the player information from the table
def parse_player_data (field_names, table):
    players = []
    data_rows = table.findAll('tr')
    for row in data_rows:
        if row.attrs['class'][0] == 'colhead':
            continue

        data_cells = row.findAll('td')

        # take index 1 (player name + position in the same column, separated by ', ') and split it into 2 fields.
        player_and_position = map(lambda sub_cell: sub_cell.string.lstrip(', '), data_cells[1].contents)

        # start collecting the rest of attributes at index 2. index 0 is rank (which we skip)
        # and index 1 we handled specially
        rest_attributes = map(lambda i: data_cells[i].string, range(2, len(data_cells)))

        res = []
        res.extend(player_and_position)
        res.extend(rest_attributes)
        player_dict = OrderedDict(zip(field_names, res))
        players.append(player_dict)
    return players

- Write a python loop to loop through the various pages and call these functions on the appropriate urls so that you can retrieve every player (rather than just the top few).

- Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them

In [4]:
def get_and_parse_espn_page_thingy (url):
    page = urlopen(url).read()
    soup = BeautifulSoup(page)
    table_el = soup.find(id='my-players-table').find('table')
    field_names = parse_column_headers(table_el)
    players = parse_player_data(field_names, table_el)

    next_page_button = soup.find('div', {'class':'jcarousel-next'})
    if 'jcarousel-next-disabled' not in next_page_button.attrs.get('class'):
        next_page_link = next_page_button.parent
        next_url = next_page_link.attrs.get('href')
        if next_url:
            players_from_next_page = get_and_parse_espn_page_thingy(next_url)
            players.extend(players_from_next_page)

    return players

In [5]:
player_scores_stats = get_and_parse_espn_page_thingy(SCORE_URL)
player_assists_stats = get_and_parse_espn_page_thingy(ASSIST_URL)

- Build dataframes for scores and assists

In [6]:
scores_keys = [k for k in player_scores_stats[0]]
scores_df = adorable.DataFrame(player_scores_stats, columns = scores_keys)

In [7]:
assists_keys = [k for k in player_assists_stats[0]]
assists_df = adorable.DataFrame(player_assists_stats, columns=assists_keys)

- Use the pandas.DataFrame.join() function to join your 2 pandas dataframes together and get a total result

In [8]:
merged = scores_df.merge(assists_df, on=['PLAYER','POSITION','TEAM'])
merged

,PLAYER,POSITION,TEAM,GP_x,MPG_x,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%,GP_y,MPG_y,AST,APG,TO,TOPG,AP48M,AST/TO
0,Stephen Curry,PG,GS,5,32.0,35.8,11.6-20.2,.574,5.6-10.8,.519,7.0-7.4,.946,5,32.0,29,5.8,8,1.6,8.7,3.63
1,Kevin Durant,SF,OKC,6,38.2,29.8,10.2-21.0,.484,2.8-6.7,.425,6.7-7.5,.889,6,38.2,16,2.7,14,2.3,3.4,1.14
2,Blake Griffin,PF,LAC,5,34.6,28.2,11.2-19.0,.589,0.0-0.4,.000,5.8-7.6,.763,5,34.6,20,4.0,10,2.0,5.5,2.00
3,Damian Lillard,PG,POR,6,36.7,27.3,10.0-21.3,.469,3.8-9.0,.426,3.5-4.0,.875,6,36.7,40,6.7,24,4.0,8.7,1.67
4,Russell Westbrook,PG,OKC,6,36.0,27.2,10.2-21.2,.480,1.8-5.3,.344,5.0-5.8,.857,6,36.0,63,10.5,32,5.3,14.0,1.97
5,Bradley Beal,SG,WSH,4,35.3,25.3,9.5-20.0,.475,3.3-7.0,.464,3.0-4.0,.750,4,35.3,7,1.8,11,2.8,2.4,0.64
6,James Harden,SG,HOU,5,39.0,23.8,6.0-20.4,.294,1.8-11.0,.164,10.0-11.8,.847,5,39.0,25,5.0,21,4.2,6.2,1.19
7,Isaiah Thomas,PG,BOS,4,30.3,23.5,7.5-18.0,.417,1.8-5.5,.318,6.8-7.3,.931,4,30.3,26,6.5,13,3.3,10.3,2.00
8,LeBron James,SF,CLE,5,33.8,22.2,9.4-19.2,.490,0.4-3.6,.111,3.0-5.4,.556,5,33.8,28,5.6,14,2.8,8.0,2.00
9,Dwyane Wade,SG,MIA,5,30.2,22.2,8.0-16.2,.494,1.0-2.0,.500,5.2-6.2,.839,5,30.2,21,4.2,15,3.0,6.7,1.40
